### Installation

Install dependencies 

In [1]:
%pip install langchain
%pip install openai
%pip install PyPDF2
%pip install faiss-cpu
%pip install boto3
%pip install tiktoken 
%pip install unstructured


[notice] A new release of pip is available: 23.0.1 -> 23.1.2
[notice] To update, run: python3.11 -m pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.

[notice] A new release of pip is available: 23.0.1 -> 23.1.2
[notice] To update, run: python3.11 -m pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.

[notice] A new release of pip is available: 23.0.1 -> 23.1.2
[notice] To update, run: python3.11 -m pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.

[notice] A new release of pip is available: 23.0.1 -> 23.1.2
[notice] To update, run: python3.11 -m pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.

[notice] A new release of pip is available: 23.0.1 -> 23.1.2
[notice] To update, run: python3.11 -m pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.

[notice] A new release of pip is a

### Load Required Packages

In [2]:
from PyPDF2 import PdfReader
from langchain.embeddings.openai import OpenAIEmbeddings
from langchain.text_splitter import CharacterTextSplitter, RecursiveCharacterTextSplitter
from langchain.vectorstores import ElasticVectorSearch, Pinecone, Weaviate, FAISS
from langchain.document_loaders import OnlinePDFLoader
from urllib.request import Request, urlopen
import io


from io import BytesIO

### Open AI Setup

In [3]:
import os

with open('../openai_secret.txt', 'r') as f:
    first_line = f.readline()
    os.environ["OPENAI_API_KEY"] = first_line    


### S3 Connection Setup

In [19]:
import boto3

# Set up the S3 client
s3 = boto3.client('s3')

bucket_name = "engageli-matan-media-manager-matan-us-east-1"
prefix = "matan/gpt-test/"

### Retrive the files from S3

In [20]:
documents = []

def getFilesFromFolder(folder_name = ""):
    prefix += folder_name    
    # List all objects in the S3 bucket location
    response = s3.list_objects_v2(Bucket=bucket_name, Prefix=prefix)

    # Get the URLs of all objects in the folder
    object_urls = []
    for obj in response['Contents']:
        object_url = f"https://{bucket_name}.s3.amazonaws.com/{obj['Key']}"
        object_urls.append(object_url)

    # Print the URLs of all objects in the folder
    documents.append(object_urls)
    
def getExampleFile(file_url):
    documents.append(file_url)

### PDF Reader

In [4]:
def readMultipleFiles():
    ## TODO
    x = 1

def readAndExtractOneFileFromLocal():
    reader = PdfReader("../assets/pre-processing/PDFs/2023HydrogenInvestorDayTranscript.pdf")

    raw_text = ''
    for i, page in enumerate(reader.pages):
        text = page.extract_text()
        if text:
            raw_text += text
    return raw_text

def get_pdf_from_url(url):
    """
    :param url: url to get pdf file
    :return: PdfFileReader object
    """
    remote_file = urlopen(Request(url)).read()
    memory_file = io.BytesIO(remote_file)
    pdf_file = PdfReader(memory_file)
    return pdf_file

def extract_text_from_pdf(pdf):
    raw_text = ''
    for i, page in enumerate(pdf.pages):
        text = page.extract_text()
        if text:
            raw_text += text
    return raw_text


In [5]:
pdf = get_pdf_from_url("https://engageli-matan-media-manager-matan-us-east-1.s3.amazonaws.com/matan/gpt-test/2023+Hydrogen+Investor+Day+Transcript+(2023.03.09).pdf")
raw_text = extract_text_from_pdf(pdf)

In [58]:
raw_text = readAndExtractOneFileFromLocal("https://engageli-matan-media-manager-matan-us-east-1.s3.amazonaws.com/matan/gpt-test/2023+Hydrogen+Investor+Day+Transcript+(2023.03.09).pdf")

TypeError: readAndExtractOneFileFromLocal() takes 0 positional arguments but 1 was given

In [6]:
# We need to split the text that we read into smaller chunks so that during information retreival we don't hit the token size limits. 

text_splitter = CharacterTextSplitter(        
    separator = "\n",
    chunk_size = 1000,
    chunk_overlap  = 0,
    length_function = len,
)
texts = text_splitter.split_text(raw_text)

In [60]:
text_splitter = RecursiveCharacterTextSplitter(        
    chunk_size = 1000,
    chunk_overlap  = 20,
    length_function = len,
)
texts = text_splitter.split_text(raw_text)

NameError: name 'RecursiveCharacterTextSplitter' is not defined

In [63]:
texts[24]

"somebody's career.\nAll right. So what does this look like in terms of execution? If you want to think about our strategy in\nthe most simple way, it looks like this, try to find every career starter and switcher in the world, go to\nconsumers, go to campuses world, the students are going to be starting when they graduate and go to\ngovernments who are trying to upskill their citizens, go to businesses who are trying to figure out how\nto skill up and reskill their employees. Then offer the best catalog of world-class content and credentials\nin a stackable format. In the Career Academy where professional certificates is turning out to be a really\nfoundational piece of our catalog. I'll talk more about that in a second. Then with the content and skills\nthat have been developed and with the credentials and projects that are evidence of knowledge and skills,\nhelp people get jobs.\nWe call this career pathways and help people get degrees. We call these Degree pathways. We're doing a"

In [7]:
# Download embeddings from OpenAI
embeddings = OpenAIEmbeddings()

In [8]:
docsearch = FAISS.from_texts(texts, embeddings)

In [9]:
from langchain.chains.question_answering import load_qa_chain
from langchain.llms import OpenAI
from langchain.chains.qa_with_sources import load_qa_with_sources_chain

In [10]:
chain = load_qa_chain(OpenAI(), chain_type="stuff")


In [76]:
query = "who is the head of investor relations?"
chain({"input_documents": texts, "question": query}, return_only_outputs=True)
docs = docsearch.similarity_search(query)
chain.run(input_documents=docs, question=query)

AttributeError: 'str' object has no attribute 'page_content'

In [11]:
query = "what are the 3 segments of the Coursera segment?"
docs = docsearch.similarity_search(query)
chain.run(input_documents=docs, question=query)

' The 3 segments of the Coursera segment are Consumer, Enterprise, and Degrees.'

In [13]:
query = "who are coursera's main compatitors?"
docs = docsearch.similarity_search(query)
chain.run(input_documents=docs, question=query)

" I don't know."

In [12]:
query = "when the adjusted EBIDA is planned to be positive?"
docs = docsearch.similarity_search(query)
chain.run(input_documents=docs, question=query)

' Coursera expects to be breakeven in 2024, and to show positive EBITDA in 2024.'

In [15]:
query = "what is the largest buisness segment?"
docs = docsearch.similarity_search(query)
chain.run(input_documents=docs, question=query)

' The largest business segment is Consumer, the B2C segment which accounts for the majority of their revenue.'